In [3]:
pip install sqlalchemy psycopg2-binary pandas

  Using cached sqlalchemy-2.0.45-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached psycopg2_binary-2.9.11-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached greenlet-3.3.0-cp313-cp313-win_amd64.whl.metadata (4.2 kB)
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached sqlalchemy-2.0.45-cp313-cp313-win_amd64.whl (2.1 MB)
Using cached psycopg2_binary-2.9.11-cp313-cp313-win_amd64.whl (2.7 MB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------------- ------------------------- 3.9/11.0 MB 25.3 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 27.9 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------  10.7/1


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

# -------------------------
# Конфигурация Postgres
# -------------------------
POSTGRES_URL = (
    f"postgresql://postgres:"
    f"postgres@"
    f"host.docker.internal:"
    f"5433/"
    f"postgres"
)

engine = create_engine(POSTGRES_URL)

# -------------------------
# Загрузка данных
# -------------------------
QUERY = """
SELECT
    origin,
    destination,
    departure_at,
    price,
    extracted_at
FROM aviasales.tickets
WHERE price IS NOT NULL;
"""

df = pd.read_sql(QUERY, engine)

print(f"Загружено строк: {len(df)}")


Загружено строк: 13908


In [6]:
avg_price_by_route = (
    df
    .groupby(["origin", "destination"])["price"]
    .mean()
    .reset_index()
    .rename(columns={"price": "avg_price"})
)

print("\n📊 Средняя цена по направлениям:")
print(avg_price_by_route)



📊 Средняя цена по направлениям:
   origin destination     avg_price
0     DXB         KUF  72219.086957
1     DXB         KZN  61933.666667
2     DXB         LED  51205.437294
3     DXB         MOW  35671.375916
4     DXB         OVB  55532.938462
5     DXB         SVX  51972.571429
6     DXB         UFA  58106.683168
7     IKT         DXB  68599.845238
8     KUF         DXB  59421.154018
9     KZN         DXB  53840.787037
10    LED         DXB  50623.242437
11    MOW         DXB  36911.464286
12    OVB         DXB  54962.763441
13    SVX         DXB  49436.000000
14    UFA         DXB  60157.609986


In [10]:
df["departure_at"] = pd.to_datetime(df["departure_at"])
df["extracted_at"] = pd.to_datetime(df["extracted_at"])

df["days_before_departure"] = (
    df["departure_at"] - df["extracted_at"]
).dt.days

best_days = (
    df
    .groupby("days_before_departure")["price"]
    .mean()
    .reset_index()
    .sort_values("price")
)

print("\n🕒 Средняя цена в зависимости от дней до вылёта:")
print(best_days.head(10))



🕒 Средняя цена в зависимости от дней до вылёта:
    days_before_departure         price
0                      -2  22768.500000
40                     38  35833.954545
39                     37  36838.480000
46                     44  36874.263158
41                     39  36958.181818
45                     43  37609.947368
6                       4  37680.197674
47                     45  37689.170732
43                     41  37889.484375
10                      8  37989.779762


лучше всего покупать билеты за 38-44 дней до вылета

In [12]:
df["purchase_weekday"] = df["extracted_at"].dt.day_name()

best_weekday = (
    df
    .groupby("purchase_weekday")["price"]
    .mean()
    .reset_index()
    .sort_values("price")
)

print("\n📅 Лучший день недели для покупки:")
print(best_weekday)



📅 Лучший день недели для покупки:
  purchase_weekday         price
0           Sunday  50443.292781
